# Return Evaluation
- return = total_pymnt_inv(LC 수수료 제외) - loan_amount / loan_amount

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.metrics import classification_report, precision_recall_curve, auc
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

import torch 
import os
import torch

import warnings
warnings.filterwarnings("ignore")

In [6]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')
print(device)

cuda


## Base

In [7]:
## Load original data
data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/Dataset/train_dataset.csv"
data = pd.read_csv(data_path, low_memory=False)

##load test data
return_data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/Dataset/return_feature_test.csv"
return_data = pd.read_csv(return_data_path)

In [8]:
return_data.head()

,last_fico_range_high,mo_sin_old_rev_tl_op,int_rate,annual_inc,dti,zip_code,installment,fico_range_low,total_rev_hi_lim,avg_cur_bal,bc_open_to_buy,revol_util,revol_bal,total_acc,total_pymnt,total_pymnt_inv,credit_history_years,loan_status
0,504.0,142.0,0.1335,140000.0,7.07,941.0,253.98,670.0,9000.0,10538.0,2946.0,0.673,6054.0,8.0,9142.877884,9142.88,12,0
1,689.0,79.0,0.2450,37000.0,34.38,349.0,290.59,690.0,16000.0,9859.0,4211.0,0.568,9089.0,24.0,12570.430000,12570.43,6,0
2,739.0,192.0,0.0999,72000.0,10.35,480.0,322.63,680.0,18800.0,10397.0,705.0,0.699,13147.0,22.0,10330.460000,10330.46,16,0
3,739.0,236.0,0.0721,72000.0,32.13,481.0,371.68,800.0,140200.0,5541.0,78841.0,0.066,9277.0,50.0,12817.155956,12817.16,20,0
4,709.0,155.0,0.0762,90000.0,19.29,283.0,405.10,690.0,53700.0,8023.0,32441.0,0.325,17459.0,46.0,14583.506490,14583.51,13,0


In [10]:
return_data[return_data['loan_status'] == 1].head()

,last_fico_range_high,mo_sin_old_rev_tl_op,int_rate,annual_inc,dti,zip_code,installment,fico_range_low,total_rev_hi_lim,avg_cur_bal,bc_open_to_buy,revol_util,revol_bal,total_acc,total_pymnt,total_pymnt_inv,credit_history_years,loan_status
10,499.0,122.0,0.1091,100000.0,17.27,921.0,392.36,670.0,19700.0,3124.0,7411.0,0.523,10311.0,21.0,8991.70,8991.70,12,1
12,499.0,95.0,0.0735,60000.0,18.46,342.0,558.68,725.0,28900.0,6175.0,13933.0,0.421,12167.0,9.0,6412.03,6412.03,8,1
14,504.0,81.0,0.1240,75000.0,16.58,906.0,852.69,725.0,47500.0,1917.0,36009.0,0.141,6676.0,22.0,6734.00,6734.00,11,1
18,519.0,81.0,0.1640,60000.0,15.72,276.0,785.00,670.0,13200.0,3788.0,3683.0,0.663,8753.0,10.0,4680.84,4680.84,7,1
20,579.0,112.0,0.1153,38000.0,26.34,846.0,277.12,660.0,9000.0,5176.0,1801.0,0.635,5713.0,10.0,4465.88,4465.88,10,1


In [12]:
## Return값이 전부 맞았다고 가정
#loan_status가 0이라고 가정한 것이 전부 맞으면 return = (total_pymnt_inv - loan_amount)
#loan_status가 1이라고 가정한 것이 전부 맞으면 return = 0일까?

# Fully Paid로 예측한 것만 투자
invested = return_data[return_data['loan_status'] == 0].copy()  # 예측이 Fully Paid

# 수익률 계산
invested['return'] = invested['total_pymnt_inv'] - invested['loan_amount']

KeyError: 'loan_amount'